In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

import subprocess


In [34]:
def splitter(test):
    tmp = test.split()
    tst = [tmp[4]]
    tst += tmp[8].split("/")[1].split("_")
    return tst

In [36]:
temp = subprocess.getoutput("ls -lah keys/80_1024*/b*")

df = pd.DataFrame(
    [splitter(line) for line in temp.split("\n")], columns=["disk","bs","poly","dim","base_log","level"])
df["disk"] = df["disk"].apply(lambda x: int(x[:-1]))

#df.head()
df.pivot_table(values="disk", index="level", columns="base_log")


base_log,3,4,5,6,7
level,,,,,
3,193,193,193,193,193
4,257,257,257,257,257
5,321,321,321,321,321
6,385,385,385,385,385
7,449,449,449,449,449
8,513,513,513,513,513
9,577,577,577,577,577


In [44]:
temp = subprocess.getoutput("ls -lah keys/80_1024*/sk*")

df = pd.DataFrame(
    [splitter(line) for line in temp.split("\n")], columns=["disk","bs","poly","dim","base_log","level"])
df["disk"] = df["disk"].apply(lambda x: float(x[:-1]))

#df.head()

df.pivot_table(values="disk", index="level", columns="base_log")


base_log,3,4,5,6,7
level,,,,,
3,2.15,2.15,2.15,2.15,2.15
4,2.15,2.15,2.15,2.15,2.15
5,2.15,2.15,2.15,2.15,2.15
6,2.15,2.15,2.15,2.15,2.15
7,2.15,2.15,2.15,2.15,2.15
8,2.15,2.15,2.15,2.15,2.15
9,2.15,2.15,2.15,2.15,2.15


In [33]:
temp = !cat boot_stats.txt

for base_log in range(3,8):
    for level in range(3,10):
        test = "keys/80_1024_1_%d_%d"%(base_log,level)
        if test not in [line.split()[0] for line in temp]:
            print("*"*80, "\n", test, "\n", "*"*80, "\n")
            !target/debug/test_boot $base_log $level 
            print("*"*80, "\n\n")
        

In [46]:
def splitter2(test):
    tmp = test.replace("NA","0").split()
    return tmp[0].split("/")[1].split("_") + [int(item) for item in tmp[1:]]

temp = !cat boot_stats.txt

[splitter2(line) for line in temp]
df2 = pd.DataFrame([splitter2(line) for line in temp], 
        columns=["bs","poly","dim","base_log","level","free", "sk_load", "bsk_load", "boot"])

df2.head()
                                                     

,bs,poly,dim,base_log,level,free,sk_load,bsk_load,boot
0,80,1024,1,5,3,5,13,22613,2893
1,80,1024,1,6,4,14,13,24416,3779
2,80,1024,1,3,3,0,19,18493,2880
3,80,1024,1,3,4,0,22,23930,3735
4,80,1024,1,3,6,8,16,35600,5067


In [47]:
df2.pivot_table(values="free", index="level", columns="base_log")


base_log,3,4,5,6,7
level,,,,,
3,0,0,5,8,11
4,0,6,10,14,18
5,5,10,15,20,21
6,8,14,20,22,21
7,11,18,23,22,21
8,14,22,23,22,21
9,17,24,23,22,21


In [48]:
tmp = df2.pivot_table(values="sk_load", index="level", columns="base_log")
print(tmp.mean().mean())
tmp


14.57142857142857


base_log,3,4,5,6,7
level,,,,,
3,19,12,13,8,19
4,22,13,15,13,8
5,1,7,23,8,13
6,16,12,19,46,18
7,1,21,17,17,7
8,1,24,14,18,10
9,8,22,29,10,6


In [53]:
tmp = (df2.pivot_table(values="bsk_load", index="level", columns="base_log")/1000.).round(1)
print(tmp.mean(axis=1).values)
tmp


[19.36 23.96 30.06 38.38 44.88 51.5  57.22]


base_log,3,4,5,6,7
level,,,,,
3,18.5,18.1,22.6,18.0,19.6
4,23.9,23.8,23.6,24.4,24.1
5,29.2,29.8,29.8,30.7,30.8
6,35.6,35.8,36.0,46.5,38.0
7,41.1,43.3,52.5,42.6,44.9
8,47.8,49.7,48.7,62.4,48.9
9,53.4,55.4,56.4,62.3,58.6


In [54]:
tmp = (df2.pivot_table(values="boot", index="level", columns="base_log")/1000.).round(1)
print(tmp.mean(axis=1).values)
tmp

[2.92 3.72 4.44 5.18 5.9  6.72 7.52]


base_log,3,4,5,6,7
level,,,,,
3,2.9,2.9,2.9,2.9,3.0
4,3.7,3.7,3.7,3.8,3.7
5,4.3,4.5,4.4,4.6,4.4
6,5.1,5.2,5.1,5.2,5.3
7,5.9,6.0,5.8,5.9,5.9
8,6.6,6.9,6.7,6.9,6.5
9,7.5,7.3,7.7,7.4,7.7


In [59]:
np.exp(-np.log(2)/(24*7))

0.9996562360941261

5.625